# Program Analysis Seedling Kickoff Demo

*University of Arizona*

*July 26, 2018*


In this demo we will show how a dynamic Bayes network (DBN) can be constructed from the source code for a FORTRAN program. This notebook has been tested with the version of Delphi corresponding to the commit hash below.

In [ ]:
!git rev-parse HEAD

Some preliminaries:

In [ ]:
%load_ext autoreload
%autoreload 2
import json
import delphi.program_analysis as pa
from IPython.core.display import Image
import delphi.jupyter_tools as jt

We start with the FORTRAN program, `crop_yield.f` shown below.

In [ ]:
jt.display(f'../data/program_analysis/crop_yield.f')

## Analyze the FORTRAN source code

The FORTRAN program is now analyzed by the autoTranslate 
program analysis pipeline:
- Analyze the FORTRAN source
- Translate the FORTRAN source analysis into a functionally equivalent Python source file
- Extract component assignment functions (lambda fns) and variable functional relationships within scope (DBN-JSON)

In [ ]:
%cd ../delphi/program_analysis/autoTranslate/
!./autoTranslate ../../../data/program_analysis/crop_yield.f
%cd ../../../notebooks/

This generates several files.  First, program analysis autoTranslate analyses the FORTRAN code to extract the AST (abstract syntax tree), represented in xml:

In [ ]:
jt.display('../delphi/program_analysis/autoTranslate/crop_yield.xml')

A functionally equivalent Python source file is generated (preserving semantics of the FORTRAN source, such as call be reference):

In [ ]:
jt.display('../delphi/program_analysis/autoTranslate/crop_yield.py')

Variable assignment functions are extracted into an interface of modular executable functions

In [ ]:
lambdas = '../delphi/program_analysis/autoTranslate/lambdas.py'
jt.display(lambdas)

The Python source is then parsed into a JSON representation of the 
statements and functions contained in the program. 

In [ ]:
dbn_json = '../delphi/program_analysis/autoTranslate/pgm.json'
jt.display(dbn_json)

## Assemble the DBN

Delphi takes this JSON description, discovers all scopes and assignment 
statements in it, and builds a graph that shows how 
information flows through the program during runtime.

### Program initialization

We construct a nested scope tree, and display the graph representation from 
the nested scope tree.

### Constructing the scope tree

The scope tree is a rooted collection of nested `Scope` objects.
We currently have two different scopes that our program identifies:

- A `LoopScope` meant to track loops found in the initial program 
- A `FuncScope` meant to track functions found in the initial program.

The `scope_tree` defined below will be a `Scope` object that has other `Scope` obejects as its children.

### Viewing the DBN

The final step is to construct a DBN from `scope_tree`. Below we create a digraph to show the DBN as a flow of information through the program at runtime. Scopes are nested and the breadth of each scope is shown with a colored bounding box. Function scopes are colored in green while loop scopes are shown in blue. Each scope is labeled with the scope specific name as found in the JSON specification. Variables from the program are shown as ellipses contained in their appropriate scopes. Variables are named with the name of the variable in the program and the name of the scope in which the variable was first defined. Actions, such as variable assignment or conditional evaluation, are shown as red rectangles and are labeled similarly to variables.

This graph has a linked structure that shows how variables from one scope populate for use into child scopes. This allows us to see the entire data flow profile of the initial program at runtime.

In [ ]:
from delphi.program_analysis.scopes import Scope
root = Scope.from_json('../delphi/program_analysis/autoTranslate/pgm.json')
A = root.to_agraph()
Image(A.draw(format='png', prog='dot'), retina=True)

## Executing the DBN

With the DBN now assembled and represented within Delphi, we can execute the DBN, displayed here from a CAG view of the DBN with variable states updating as the loop is unfolded.

In [ ]:
from delphi.program_analysis.ProgramAnalysisGraph import *
import sys
from importlib import import_module
sys.path.append('../delphi/program_analysis/autoTranslate')
import lambdas
G = ProgramAnalysisCAG.from_agraph(A, lambdas)

In [ ]:
initialize(G)
G.visualize(show_values = True)

In [ ]:
update(G)
G.visualize(show_values = True)